
In their second Quantum Open Science Prize, IBM offers $100,000 in prize money to those whose algorithms can best handle the inherent noise of their real Jakarta-based quantum computer. Since the Jakarta device is too small to apply quantum error correction codes, it won't be able to achieve a perfect result. Therefore, the best we can do is to use a quantum error mitigation method that reduces how the noise affects the result of a computation.

in the previous post, we saw that we can use CDR to effectively reduce the impact noise has on the result of a the exemplary quantum computation. We reduced the errors by 60%. 

So, let's take the next step towards solving the IBM challenge by not running the Mitiq example but the Trotterized Heisenberg Simulation.

First revisit the code we ran.

The quantum circuit we need to define represents the problem we aim to solve, such as the Hamiltonian simulation IBM asks us for. Yet, we stick with the example Mitiq provides. This is a two-qubit circuit that only consists of Clifford gates and rotations around the Z-axis ( 𝑅𝑍 ). Clifford gates are easy to simulate on a classical computer--a precondition for the CDR method.


In [9]:
# 1. Define a quantum circuit
from qiskit import QuantumCircuit

def get_circuit():
    qc = QuantumCircuit(2)

    # CDR works better if the circuit is not too short. So we increase its depth.
    for i in range(5): 
        qc.h(0) # Clifford
        qc.h(1) # Clifford
        qc.rz(1.75, 0)
        qc.rz(2.31, 1)
        qc.cx(0,1) # Clifford
        qc.rz(-1.17, 1)
        qc.rz(3.23, 0)
        qc.rx(pi/2, 0) # Clifford
        qc.rx(pi/2, 1) # Clifford

    # We need to measure the qubits
    #qc.measure_all()
    return qc

Generally, the observable is something we can measure. But, let's not get into the physical details too much. Rather, let's look at it from a conceptual perspective.

A qubit is a two-dimensional system as depicted in the following image. The poles of the visualization depict the basis states  |0⟩  and  |1⟩ . The arrow is the quantum state vector. The proximities to the poles (the basis states) denote the amplitudes whose squares are the probabilities of measuring the qubit as either 0 or 1. Simply put, the closer the quantum state vector is to the basis state  |1⟩  the higher the probability of measuring the qubit as a 1.

In [10]:
from mitiq import Observable, PauliString

obs = Observable(PauliString("ZZ"))
print(obs)

Z(0)*Z(1)


In [ ]:
And, we need to executors. A noisy and a noise-free.

In [11]:
# 2. Define an executor
from mitiq.interface.mitiq_qiskit import qiskit_utils
from qiskit import QuantumCircuit, execute, Aer
import qiskit.providers.aer.noise as noise

# Error probabilities
prob_1 = 0.005  # 1-qubit gate
prob_2 = 0.01   # 2-qubit gate

# Depolarizing quantum errors
error_1 = noise.depolarizing_error(prob_1, 1)
error_2 = noise.depolarizing_error(prob_2, 2)

# Add errors to noise model
noise_model = noise.NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

def sim_noise(qc):
    return qiskit_utils.execute_with_shots_and_noise(qc, obs.matrix(), noise_model, 4096)


In [12]:
def sim(qc):
    return qiskit_utils.execute_with_shots(qc, obs.matrix(), 4096)


In [13]:
ideal_measurement = obs.expectation(get_circuit(), sim).real
print("ideal_measurement = ",ideal_measurement)

ideal_measurement =  0.98388671875


Then, we compute the unmitigated noisy result.


In [14]:
unmitigated_measurement = obs.expectation(get_circuit(), sim_noise).real
print("unmitigated_measurement = ", unmitigated_measurement)

unmitigated_measurement =  0.85888671875


In [ ]:
So, let's use the Trotterized circuit

In [15]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})  # enlarge matplotlib fonts

# Import qubit states Zero (|0>) and One (|1>), and Pauli operators (X, Y, Z)
from qiskit.opflow import Zero, One, I, X, Y, Z
from qiskit import QuantumCircuit, QuantumRegister, IBMQ, execute, transpile, Aer
from qiskit.providers.aer import QasmSimulator
from qiskit.tools.monitor import job_monitor
from qiskit.circuit import Parameter

# Import state tomography modules
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

from qiskit import QuantumCircuit
from mitiq.interface.mitiq_qiskit import qiskit_utils
from mitiq import Observable, PauliString
from typing import Any, Callable, Optional, Sequence, Union
import numpy as np
from scipy.optimize import curve_fit

from mitiq import Executor, Observable, QPROGRAM, QuantumResult
from mitiq.cdr import (
    generate_training_circuits,
    linear_fit_function,
    linear_fit_function_no_intercept,
    is_clifford,
)
from mitiq.zne.scaling import fold_gates_at_random


# YOUR TROTTERIZATION GOES HERE -- START (beginning of example)

# Parameterize variable t to be evaluated at t=pi later
t = Parameter('t')

# Build a subcircuit for XX(t) two-qubit gate
XX_qr = QuantumRegister(2)
XX_qc = QuantumCircuit(XX_qr, name='XX')

XX_qc.ry(np.pi/2,[0,1])
XX_qc.cnot(0,1)
XX_qc.rz(2 * t, 1)
XX_qc.cnot(0,1)
XX_qc.ry(-np.pi/2,[0,1])

# Convert custom quantum circuit into a gate
XX = XX_qc.to_instruction()

# Build a subcircuit for YY(t) two-qubit gate
YY_qr = QuantumRegister(2)
YY_qc = QuantumCircuit(YY_qr, name='YY')

YY_qc.rx(np.pi/2,[0,1])
YY_qc.cnot(0,1)
YY_qc.rz(2 * t, 1)
YY_qc.cnot(0,1)
YY_qc.rx(-np.pi/2,[0,1])

# Convert custom quantum circuit into a gate
YY = YY_qc.to_instruction()

# Build a subcircuit for ZZ(t) two-qubit gate
ZZ_qr = QuantumRegister(2)
ZZ_qc = QuantumCircuit(ZZ_qr, name='ZZ')

ZZ_qc.cnot(0,1)
ZZ_qc.rz(2 * t, 1)
ZZ_qc.cnot(0,1)

# Convert custom quantum circuit into a gate
ZZ = ZZ_qc.to_instruction()

# Combine subcircuits into a single multiqubit gate representing a single trotter step
num_qubits = 3

Trot_qr = QuantumRegister(num_qubits)
Trot_qc = QuantumCircuit(Trot_qr, name='Trot')

for i in range(0, num_qubits - 1):
    Trot_qc.append(ZZ, [Trot_qr[i], Trot_qr[i+1]])
    Trot_qc.append(YY, [Trot_qr[i], Trot_qr[i+1]])
    Trot_qc.append(XX, [Trot_qr[i], Trot_qr[i+1]])

# Convert custom quantum circuit into a gate
Trot_gate = Trot_qc.to_instruction()
XX_qc.draw()
# YOUR TROTTERIZATION GOES HERE -- FINISH (end of example)

def get_circuit(steps=4):
    
    # The final time of the state evolution
    target_time = np.pi

    # Number of trotter steps
    trotter_steps = steps  ### CAN BE >= 4

    # Initialize quantum circuit for 3 qubits
    qr = QuantumRegister(7)
    qc = QuantumCircuit(qr)

    # Prepare initial state (remember we are only evolving 3 of the 7 qubits on jakarta qubits (q_5, q_3, q_1) corresponding to the state |110>)
    qc.x([3,5])  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)

    # Simulate time evolution under H_heis3 Hamiltonian
    for _ in range(trotter_steps):
        qc.append(Trot_gate, [qr[1], qr[3], qr[5]])

    # Evaluate simulation at target_time (t=pi) meaning each trotter step evolves pi/trotter_steps in time
    qc = qc.bind_parameters({t: target_time/trotter_steps})

    # Generate state tomography circuits to evaluate fidelity of simulation
    st_qcs = state_tomography_circuits(qc, [qr[1], qr[3], qr[5]])

    
    
    return st_qcs[-1]

get_circuit().draw()  # only view trotter gates

/tmp/ipykernel_61/2156713432.py:13: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter


░       »
q23_0: ──────────────────────────────────────────────────────────────░───────»
            ┌────────────┐┌────────────┐┌────────────┐┌────────────┐ ░ ┌─┐   »
q23_1: ─────┤0           ├┤0           ├┤0           ├┤0           ├─░─┤M├───»
            │            ││            ││            ││            │ ░ └╥┘   »
q23_2: ─────┤            ├┤            ├┤            ├┤            ├─░──╫────»
       ┌───┐│            ││            ││            ││            │ ░  ║ ┌─┐»
q23_3: ┤ X ├┤1 Trot(π/4) ├┤1 Trot(π/4) ├┤1 Trot(π/4) ├┤1 Trot(π/4) ├─░──╫─┤M├»
       └───┘│            ││            ││            ││            │ ░  ║ └╥┘»
q23_4: ─────┤            ├┤            ├┤            ├┤            ├─░──╫──╫─»
       ┌───┐│            ││            ││            ││            │ ░  ║  ║ »
q23_5: ┤ X ├┤2           ├┤2           ├┤2           ├┤2           ├─░──╫──╫─»
       └───┘└────────────┘└────────────┘└────────────┘└────────────┘ ░  ║  ║ »
q23_6: ──────────────────────────────────────────────────────────────░──╫──╫─»
                                                                     ░  ║  ║ »
 c0: 3/═════════════════════════════════════════════════════════════════╩══╩═»
                                                                        0  1 »
«          
«q23_0: ───
«          
«q23_1: ───
«          
«q23_2: ───
«          
«q23_3: ───
«          
«q23_4: ───
«       ┌─┐
«q23_5: ┤M├
«       └╥┘
«q23_6: ─╫─
«        ║ 
« c0: 3/═╩═
«        2

In [48]:
ideal_measurement = obs.expectation(get_circuit(), sim).real
print("ideal_measurement = ",ideal_measurement)

CircuitConversionError: Circuit could not be converted to an internal Mitiq circuit. This may be because the circuit contains custom gates or Pragmas (pyQuil). If you think this is a bug or that this circuit should be supported, you can open an issue at https://github.com/unitaryfund/mitiq. 

Provided circuit has type <class 'qiskit.circuit.quantumcircuit.QuantumCircuit'> and is:

                                                                     ░       »
q80_0: ──────────────────────────────────────────────────────────────░───────»
            ┌────────────┐┌────────────┐┌────────────┐┌────────────┐ ░ ┌─┐   »
q80_1: ─────┤0           ├┤0           ├┤0           ├┤0           ├─░─┤M├───»
            │            ││            ││            ││            │ ░ └╥┘   »
q80_2: ─────┤            ├┤            ├┤            ├┤            ├─░──╫────»
       ┌───┐│            ││            ││            ││            │ ░  ║ ┌─┐»
q80_3: ┤ X ├┤1 Trot(π/4) ├┤1 Trot(π/4) ├┤1 Trot(π/4) ├┤1 Trot(π

In [ ]:
We get an error. 

```
CircuitConversionError: Circuit could not be converted to an internal Mitiq circuit. This may be because the circuit contains custom gates or Pragmas (pyQuil).
```

Yes, we used custom gates.

Let's rewrite the circuit. Further, the Jakarta system has 7 qubits. But we only use three. To speed up our simulation. we make the circuit use any number of qubits.


In [26]:
def add_XX(qc, t, i):
    qc.ry(np.pi/2,[0+2*i,1+2*i])
    qc.cnot(0+2*i,1+2*i)
    qc.rz(2 * t, 1+2*i)
    qc.cnot(0+2*i,1+2*i)
    qc.ry(-np.pi/2,[0+2*i,1+2*i])
    
def add_YY(qc, t, i):
    qc.rx(np.pi/2,[0+2*i,1+2*i])
    qc.cnot(0+2*i,1+2*i)
    qc.rz(2 * t, 1+2*i)
    qc.cnot(0+2*i,1+2*i)
    qc.rx(-np.pi/2,[0+2*i,1+2*i])
    
def add_ZZ(qc, t, i):
    qc.cnot(0+2*i,1+2*i)
    qc.rz(2 * t, 1+2*i)
    qc.cnot(0+2*i,1+2*i)
    

def add_trotter_step(qc, t):
    
    # Combine subcircuits into a single multiqubit gate representing a single trotter step
    num_qubits = 3

    Trot_qr = QuantumRegister(num_qubits)
    Trot_qc = QuantumCircuit(Trot_qr, name='Trot')

    for i in range(0, num_qubits - 1):
        add_ZZ(qc, t, 1+i)
        add_YY(qc, t, 1+i)
        add_XX(qc, t, 1+i)
        
def get_circuit(steps=4):
    
    # The final time of the state evolution
    target_time = np.pi

    # Number of trotter steps
    trotter_steps = steps  ### CAN BE >= 4

    # Initialize quantum circuit for 3 qubits
    #qr = QuantumRegister(7)
    qc = QuantumCircuit(7)

    # Prepare initial state (remember we are only evolving 3 of the 7 qubits on jakarta qubits (q_5, q_3, q_1) corresponding to the state |110>)
    qc.x([3,5])  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)

    # Simulate time evolution under H_heis3 Hamiltonian
    for _ in range(trotter_steps):
        add_trotter_step(qc, target_time/trotter_steps)
        #qc.append(Trot_gate, [qr[1], qr[3], qr[5]])

    # Evaluate simulation at target_time (t=pi) meaning each trotter step evolves pi/trotter_steps in time
    #qc = qc.bind_parameters({t: })

    # Generate state tomography circuits to evaluate fidelity of simulation
    st_qcs = state_tomography_circuits(qc, [1,3,5])

    return qc
    #return st_qcs[0]


#get_circuit().draw()  # only view trotter gates

In [ ]:
We also need to change the Observable. We look at the qubits from the standard Z-basis on all qubits.

In [29]:


obs = Observable(PauliString("ZZZZ"))

In [30]:
ideal_measurement = obs.expectation(get_circuit(), sim).real
print("ideal_measurement = ",ideal_measurement)

ideal_measurement =  1.0


In [24]:
unmitigated_measurement = obs.expectation(get_circuit(), sim_noise).real
print("unmitigated_measurement = ", unmitigated_measurement)

unmitigated_measurement =  -0.00341796875


In [25]:
from mitiq import cdr

mitigated_measurement = cdr.execute_with_cdr(
    get_circuit(),
    sim_noise,
    observable=obs.matrix(),
    simulator=sim,
    seed=0,
).real
print("mitigated_measurement = ", mitigated_measurement)

mitigated_measurement =  0.0048828125


In [ ]:
Maybe because the circuit consists only of Clifford gates?
But we only used a single state tomography circuit. But there are multiple needed.